# ETL User Items

In [1]:
import json
import os
import pandas as pd
import gzip
import ast
import numpy as np
import re
from datetime import datetime

In [2]:
rows = []

# Open the gzip-compressed JSON file
with gzip.open(r'C:\Users\flore\OneDrive\Escritorio\Etapa Labs\MLOPs\01. PI MLOps - STEAM\users_items.json.gz', 'rb') as f:
    # Iterate over each line in the file
    for line in f.readlines():
        # Decode the line and evaluate it as a Python literal
        rows.append(ast.literal_eval(line.decode('utf-8')))
        
# Convert the list of dictionaries into a DataFrame
df_items = pd.DataFrame(rows)

# Display the first few rows of the DataFrame
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [3]:
df_items.columns

Index(['user_id', 'items_count', 'steam_id', 'user_url', 'items'], dtype='object')

In [4]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


### Check for duplicates
Before me move into unnesting the column items, we check for duplicates in the column 'user_id'.

In [5]:
# Sort the DataFrame by 'user_id'
df_sorted = df_items.sort_values(by='user_id')

# Check for duplicate rows based on 'user_id' column
duplicated_rows = df_sorted[df_sorted.duplicated(subset=['user_id'], keep=False)]
duplicated_rows.head(20)

,user_id,items_count,steam_id,user_url,items
29193,05041129,35,76561198167088451,http://steamcommunity.com/id/05041129,"[{'item_id': '4000', 'item_name': 'Garry's Mod..."
11000,05041129,35,76561198167088451,http://steamcommunity.com/id/05041129,"[{'item_id': '4000', 'item_name': 'Garry's Mod..."
37062,10outof10matee,56,76561198050688208,http://steamcommunity.com/id/10outof10matee,"[{'item_id': '220', 'item_name': 'Half-Life 2'..."
37061,10outof10matee,56,76561198050688208,http://steamcommunity.com/id/10outof10matee,"[{'item_id': '220', 'item_name': 'Half-Life 2'..."
6163,111222333444555666888,52,76561198082607692,http://steamcommunity.com/id/11122233344455566...,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
6167,111222333444555666888,52,76561198082607692,http://steamcommunity.com/id/11122233344455566...,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
29108,128bitbit,81,76561198061104290,http://steamcommunity.com/id/128bitbit,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
29107,128bitbit,81,76561198061104290,http://steamcommunity.com/id/128bitbit,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
19777,13272310,28,76561198087622450,http://steamcommunity.com/id/13272310,"[{'item_id': '57690', 'item_name': 'Tropico 4'..."
19774,13272310,28,76561198087622450,http://steamcommunity.com/id/13272310,"[{'item_id': '57690', 'item_name': 'Tropico 4'..."


#### All of these registers: 1357 in total are clearly duplicated. So we will proceed to remove the duplicates and keep the first occurance<br>

In [6]:
# Drop duplicated rows based on 'user_id' and 'user_url' columns, keeping the first occurrence
df_items = df_items.drop_duplicates(subset=['user_id'], keep='first')

# Display the DataFrame after dropping duplicates
df_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [5]:
df_items['user_id'].nunique()

87626

### This is the total amount of users in the platform

### We review the last column, number 4, that has for each user_id the list of items owned by the user.

In [7]:
df_items.iloc[10,4]

[{'item_id': '4000',
  'item_name': "Garry's Mod",
  'playtime_forever': 2644,
  'playtime_2weeks': 0},
 {'item_id': '1250',
  'item_name': 'Killing Floor',
  'playtime_forever': 30266,
  'playtime_2weeks': 0},
 {'item_id': '35420',
  'item_name': 'Killing Floor Mod: Defence Alliance 2',
  'playtime_forever': 54,
  'playtime_2weeks': 0},
 {'item_id': '6060',
  'item_name': 'STAR WARS™ Battlefront™ II',
  'playtime_forever': 86,
  'playtime_2weeks': 0},
 {'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 108,
  'playtime_2weeks': 0},
 {'item_id': '20',
  'item_name': 'Team Fortress Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '30',
  'item_name': 'Day of Defeat',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '40',
  'item_name': 'Deathmatch Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '50',
  'item_name': 'Half-Life: Opposing Force',
  'playtime_forever': 45,
  'playtime_2weeks': 0},
 {'item_i

#### In order to make this information useful we need to convert the information contain in the column into columns item_id, item_name, playtime_forever, ply time_2weeks.  

In [8]:
# unnest the json objects in the column items
items_des = []

for index, row in df_items.iterrows():
    user_id = row['user_id']
    items_count = row['items_count']
    steam_id = row['steam_id']
    user_url = row['user_url']
    items = row['items']
    
    for i in items:   
        new_row = {
        'user_id': user_id,
        'items_count': items_count,
        'steam_id' : steam_id,
        'user_url' : user_url,
        'item_id': i.get('item_id', ''),
        'item_name': i.get('item_name', ''),
        'playtime_forever': i.get('playtime_forever', ''),
        'playtime_2weeks': i.get('playtime_2weeks', '')
        }
        
        items_des.append(new_row)

df_items_unnested = pd.DataFrame(items_des)

In [10]:
df_items_unnested.head(4)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0,0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7,0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0,0


In [9]:
df_items_unnested.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094082 entries, 0 to 5094081
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       int64 
 2   steam_id          object
 3   user_url          object
 4   item_id           object
 5   item_name         object
 6   playtime_forever  int64 
 7   playtime_2weeks   int64 
dtypes: int64(3), object(5)
memory usage: 310.9+ MB


In [11]:
df_items = df_items_unnested

In [12]:
# Count the number of missing values in each column
missing_count = df_items.isna().sum()
# Calculate the percentae of missing values for each column
missing_percentage = (missing_count / len(df_items)) * 100
# Combine the missing cont and percentage into a DataFrame
missing_info = pd.DataFrame({'Missing Count': missing_count, 'Missing Percentage': missing_percentage})
# Add a new column for the total number of rows in the DataFrame
missing_info['Total Registers'] = len(df_items)
# Reorder columns
missing_info = missing_info[['Total Registers', 'Missing Count', 'Missing Percentage']]
# Display the missing information
missing_info

,Total Registers,Missing Count,Missing Percentage
user_id,5094082,0,0.0
items_count,5094082,0,0.0
steam_id,5094082,0,0.0
user_url,5094082,0,0.0
item_id,5094082,0,0.0
item_name,5094082,0,0.0
playtime_forever,5094082,0,0.0
playtime_2weeks,5094082,0,0.0


### Valuable information and disposable data
Now that we have some insight into the dataframe structure and information we can conclude that there are many columns we can drop in order to keep our data manageable and as compact as possible<br>
#### Columns to drop:<br>
-   **items_count** is a column that given the information can be calculated by request<br>
-   **steam_id** is the same exact value as **user_id**
-   **user_url** it is already contained in the review dataframe so in case it is needed it can be then pulled from that dataframe by request<br>

In [13]:
df_items.drop(['items_count','steam_id','user_url'], axis=1, inplace= True)

### Correct datatypes for columns 'user_id' and ' item_id'

In [14]:
df_items["item_id"] = pd.to_numeric(df_items["item_id"], errors="coerce")

In [15]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094082 entries, 0 to 5094081
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   item_id           int64 
 2   item_name         object
 3   playtime_forever  int64 
 4   playtime_2weeks   int64 
dtypes: int64(3), object(2)
memory usage: 194.3+ MB


### Store items Dataframe 
Now that we have done the load and transformation of the data into a valueable information we store it to proceed with the EDA.
We choose to store the data as .parquet beacuse of the size limitations<br>

In [17]:
# Define the file path for storing the Parquet file
items = 'data/items.parquet'

# Store the DataFrame as a Parquet file
df_items.to_parquet(items, index=False)

# Print a message confirming the storage location
print(f'Items DataFrame was stored into {items}')

Items DataFrame was stored into data/items.parquet
